In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("PRSS_run_name", "")
PRSS_run_name = dbutils.widgets.get("PRSS_run_name")

export_all_output = False
debug = True

In [0]:
import re
import os
import glob
import shutil
import datetime as dt
import pytz

prss_folder = f"/Volumes/ea_prd/sourcefiles/datalake/vspd/gdx/PRSS/"

if PRSS_run_name == "":
  # Set New Zealand timezone
  nz_tz = pytz.timezone("Pacific/Auckland")
  # Get current datetime in NZ time
  now = dt.datetime.now(nz_tz)
  # Round down to the last 30-minute mark
  rounded = now.replace(minute=(now.minute // 30) * 30, second=0, microsecond=0).strftime("%Y%m%d%H%M")
  # Check if the PRSS run name exists
  yyyymmdd = now.strftime("%Y%m%d")
  pattern = f"{prss_folder}{yyyymmdd}/PRSS_{rounded}_*.gdx"
  files = glob.glob(pattern)
  if files:
    PRSS_file = files[0]
  else:
    dbutils.notebook.exit("File does not exist") 
else:
  # Extract the date part (yyyymmdd)
  date_part = PRSS_run_name[5:13]  # Extracts the "yyyymmdd" part from the filename
  # Create the new file path
  PRSS_file = f"{prss_folder}{date_part}/{PRSS_run_name}.gdx"

if PRSS_file:
  print("File exists:", PRSS_file)  # Print the first matching file
else:
  dbutils.notebook.exit("File does not exist")


In [0]:
# src = "/Volumes/ea_dev/sourcefiles/datalake/vSPD"
src = "/Workspace/Users/tuong.nguyen@ea.govt.nz/vSPD"
dst = "/tmp/vSPD"
shutil.copytree(src, dst,dirs_exist_ok=True)
if debug: os.listdir('/tmp/vSPD')

In [0]:
src_folder = PRSS_file.replace(".gdx","")
dst_folder = "/tmp/vSPD/Programs/"

# Ensure the destination folder exists
os.makedirs(dst_folder, exist_ok=True)

# Loop through all files in the source folder
for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)
    
    # If it's a file (not a directory), copy it
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)

if debug: print("\n".join(os.listdir('/tmp/vSPD/Programs/')))

In [0]:
file_path = '/tmp/vSPD/Programs/vSPDsettings.inc'
with open(file_path, "r") as file: content = file.read().replace("\\", "/")
with open(file_path, "w") as file: file.write(content)
if debug: os.system("cat /tmp/vSPD/Programs/vSPDsettings.inc")

In [0]:
dst_file = os.path.join("/tmp/vSPD/Input/", os.path.basename(PRSS_file))
shutil.copy(PRSS_file, dst_file)
if debug: 
  print("\n".join(os.listdir("/tmp/vSPD/Input/")))

In [0]:
import subprocess

programs_dir = '/tmp/vSPD/Programs'
os.chdir(programs_dir)

# Path to your GAMS executable
gams_path = '/dbfs/GAMS/gams48.6_linux_x64_64_sfx/gams'

# Command to run
command = f"{gams_path} runvSPD.gms"

# Run the command using subprocess
subprocess.run(command, shell=True, check=True, capture_output=True)

In [0]:
# Copy NodePriceSensitivity output file to destination

src_file = f"/tmp/vSPD/Output/{os.path.basename(src_folder)}/{os.path.basename(src_folder)}_NodePriceSensitivity.csv"
dst_folder = f"/Volumes/ea_dev/sourcefiles/datalake/vSPDDemandPriceSensitivityNode/"
os.makedirs(dst_folder, exist_ok=True)
dst_file = f"{dst_folder}{os.path.basename(src_folder)}_NodePriceSensitivity.csv"
shutil.copy(src_file, dst_file)

# Copy IslandSensitivity output file to destination
src_file = f"/tmp/vSPD/Output/{os.path.basename(src_folder)}/{os.path.basename(src_folder)}_IslandSensitivity.csv"
dst_folder = f"/Volumes/ea_dev/sourcefiles/datalake/vSPDDemandPriceSensitivityIsland/"
os.makedirs(dst_folder, exist_ok=True)
dst_file = f"{dst_folder}{os.path.basename(src_folder)}_IslandSensitivity.csv"
shutil.copy(src_file, dst_file)


In [0]:
dbutils.notebook.exit("Done")

#TEST CODE